# Corporacion Favorita - New Superb Forecasting Model - 

## Split and Model Pipeline

#codi

Made by 4B Consultancy (Janne Heuvelmans, Georgi Duev, Alexander Engelage, Sebastiaan de Bruin) - 2024

In this data pipeline, 

The following steps are made within this notebook:  

>-0. Import Packages 

>-1. Load final dataset and aggregate dataset to weekly level
    -1.1 Load final dataset made in Data Preperation Pipeline Notebook
    -1.2 Aggregate dataset to weekly level

>-2. Column transformers and Train, Test, Validation Split

>-3. Models

>-4. Pick best model one and optimize with grid search

## 0. Import Packages

In [2]:
# Importing the libraries
import pandas as pd
import numpy as np
import polars as pl
import os
import sys
import altair as alt
import vegafusion as vf
import sklearn
import time
from datetime import date, datetime, timedelta
from sklearn.pipeline import Pipeline, make_pipeline

In [3]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_absolute_percentage_error

import statsmodels.api as sm

In [4]:
from sklearn.model_selection import train_test_split

## 1. Load final dataset, Inpute Stockouts and Aggregate dataset to weekly level

### 1.1. Functions - Import raw data from local PATH
Create import data function and give basic information function within the importing function.

Return basic information on each dataframe:  
- a) Information on the number of observation and features.  
- b) Information on the size of the dataframe. 

TO-DO: Import via polars, and use polars dataframe?

In [5]:
def f_get_data_and_info(import_path, file_name="df_final"):

    print(f"\nReading file {file_name}\n")

    # Load data.
    df = pd.read_parquet(import_path + file_name + ".parquet")

    # Getting the basic information of the dataframe (number of observations and features, and size)
    print(
        f"The '{file_name}' dataframe contains: {df.shape[0]:,}".replace(",", ".")
        + f" observations and {df.shape[1]} features."
    )
    print(
        f"Prepared and transformed dataframe has optimized size of {round(sys.getsizeof(df)/1024/1024/1024, 2)} GB."
    )

    return df

### 1.2. Importing raw data
Importing parquet files with importing function (giving basic information)

In [6]:
import_path = "C:/Users/alexander/Documents/0. Data Science and AI for Experts/EAISI_4B_Supermarket/data/processed/"


# import_path = "C:/Users/sebas/OneDrive/Documenten/GitHub/Supermarketcasegroupproject/Group4B/data/raw/


# Importing final df


df_final = f_get_data_and_info(import_path, file_name="Prepped_data_20241004")


Reading file Prepped_data_20241004

The 'Prepped_data_20241004' dataframe contains: 67.834.270 observations and 19 features.
Prepared and transformed dataframe has optimized size of 2.72 GB.


To-do: include null_count print dunction into importing OR make basic descrption function with features, size, null_count

In [7]:
df_final.info()
# Count nulls per column
null_counts = df_final.isnull().sum()

# Print results
for column, count in null_counts.items():
    print(f"Column '{column}' has {count} null values.")

<class 'pandas.core.frame.DataFrame'>
Index: 67834270 entries, 0 to 67834269
Data columns (total 19 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   store_nbr               uint8         
 1   item_nbr                int32         
 2   date                    datetime64[ns]
 3   unit_sales              float32       
 4   onpromotion             bool          
 5   holiday_local_count     int8          
 6   holiday_national_count  int8          
 7   holiday_regional_count  int8          
 8   store_type              category      
 9   store_cluster           uint8         
 10  item_family             category      
 11  item_class              uint16        
 12  perishable              uint8         
 13  store_status            int8          
 14  item_status             int8          
 15  year                    int16         
 16  weekday                 int8          
 17  week_nbr                int8          
 18  week_

## 2.0 Train test val split

SKtime

ExpandingWindowSplitter


#TO-DO: Selecting on weeks or via data?

In [8]:
features = [
    "store_nbr",
    "item_nbr",
    "onpromotion",
    "holiday_local_count",
    "holiday_national_count",
    "holiday_regional_count",
    "store_type",
    "store_cluster",
    "item_family",
    "item_class",
    "perishable",
    "store_status",
    "item_status",
    "year",
    # "week_nbr",
    "week_number_cum",
]

target_variable = ["unit_sales"]

In [9]:
def train_val_test_split(df, features, target_variable, window_length=26):

    # Sort the DataFrame by store number, item number, and date for ordering
    df = df.sort_values(["store_nbr", "item_nbr", "week_number_cum"])

    # Create X (features) and y (target)
    X = df[features]
    y = df[target_variable]

    # Get the maximum week in the dataset
    max_week = df["week_number_cum"].max()

    # Calculate start and end weeks for validation and test sets
    test_week_start = max_week - window_length + 1

    val_week_start = max_week - 2 * window_length + 1

    val_week_end = test_week_start - 1

    train_week_end = val_week_start - 1

    # 1. Train data: All data before the start of the validation period
    X_train = X[X["week_number_cum"] <= train_week_end]
    y_train = y[df["week_number_cum"] <= train_week_end]

    # 2. Val data: From `val_week_start` to `val_week_end`
    X_val = X[
        (X["week_number_cum"] >= val_week_start)
        & (X["week_number_cum"] <= val_week_end)
    ]
    y_val = y[
        (df["week_number_cum"] >= val_week_start)
        & (df["week_number_cum"] <= val_week_end)
    ]

    # 3. Test data: From `test_week_start` to `max_week`
    X_test = X[
        (X["week_number_cum"] >= test_week_start) & (X["week_number_cum"] <= max_week)
    ]
    y_test = y[
        (df["week_number_cum"] >= test_week_start) & (df["week_number_cum"] <= max_week)
    ]

    # Function to print split information
    def print_split_info(split_name, X_split, y_split):
        print(f"\n{split_name} set:")
        print(f"X_{split_name.lower()} shape: {X_split.shape}")
        print(f"y_{split_name.lower()} shape: {y_split.shape}")
        print(f"{split_name} Min Week: {X_split['week_number_cum'].min()}")
        print(f"{split_name} Max Week: {X_split['week_number_cum'].max()}")
        print(f"{split_name} number of weeks: {X_split['week_number_cum'].nunique()}")
        print(f"Number of stores: {X_split['store_nbr'].nunique()}")
        print(f"Number of items: {X_split['item_nbr'].nunique()}")

    # Print information about the splits
    print_split_info("Train", X_train, y_train)
    print_split_info("Validation", X_val, y_val)
    print_split_info("Test", X_test, y_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [10]:
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(
    df_final, features, target_variable, window_length=26
)


Train set:
X_train shape: (53398880, 15)
y_train shape: (53398880, 1)
Train Min Week: 1
Train Max Week: 190
Train number of weeks: 190
Number of stores: 10
Number of items: 4021

Validation set:
X_validation shape: (7318220, 15)
y_validation shape: (7318220, 1)
Validation Min Week: 191
Validation Max Week: 216
Validation number of weeks: 26
Number of stores: 10
Number of items: 4021

Test set:
X_test shape: (7117170, 15)
y_test shape: (7117170, 1)
Test Min Week: 217
Test Max Week: 242
Test number of weeks: 26
Number of stores: 10
Number of items: 4021


To-do: do we split based on dates or based on weeks since start?

In [11]:
# X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(
#     df_final, train_end="2016-06-01", test_end="2017-01-01"
# )

## 3.0 Functions - Impute stockouts and Aggregate dataset to weekly level


#### 3.1. Impute stockouts

Stockout on store level

•      Perishable good: when there are missing values for two consecutive days for a given item per individual store 

•      Nonperishable goods: when there are missing values for 7 consecutive days for a given item and per individual store

•      Action: Impute with Rolling Mean with defeault window of 7 days 

------------------------------------

In [12]:
def impute_stockouts_polars(df_pandas, window_size=7):

    # Convert the input Pandas DataFrame to a Polars DataFrame
    df = pl.from_pandas(df_pandas)

    # Sort the DataFrame by store number, item number, and date for ordering

    df = df.sort(["store_nbr", "item_nbr", "date"])

    # Nested function calc_missing_count to calculate the count of consecutive missing values in unit_sales

    def calc_missing_count(unit_sales):

        return (
            unit_sales.is_null()  # Check for null values
            .cast(pl.Int32)  # Cast to integer (1 for null, 0 for not null)
            .cum_sum()  # Cumulative sum to count sequential nulls
            .over(["store_nbr", "item_nbr"])  # Group by store_nbr and item_nbr
        )

    # Nested function to Inpute with rolling mean for missing values
    def rolling_mean_imputation(unit_sales, window_size):

        return (
            unit_sales.rolling_mean(
                window_size=window_size, min_periods=1
            )  # Impute strategy based on rolling mean
            .shift(
                1
            )  # Shift window by one day, to prevent taking the same day into account
            .over(["store_nbr", "item_nbr"])  # Group by store_nbr and item_nbr
        )

    # Apply the imputation logic based on the perishable status of the items

    df = df.with_columns(
        [
            pl.when(pl.col("perishable") == 1)  # Check if the item is perishable = 1
            .then(
                pl.when(
                    calc_missing_count(pl.col("unit_sales")) == 1
                )  # 1 missing value
                .then(0)  # --> Impute with 0
                .when(
                    calc_missing_count(pl.col("unit_sales")) > 2
                )  # More than 2 missing values
                .then(0)  # --> Impute with 0
                .when(
                    calc_missing_count(pl.col("unit_sales")) == 2
                )  # = 2 missing values
                .then(
                    rolling_mean_imputation(pl.col("unit_sales"), window_size)
                )  # --> Inpute with rolling mean for 2 missing days
                .otherwise(pl.col("unit_sales"))  # Otherwise keep original value
            )
            .when(pl.col("perishable") == 0)  # If the item is not perishable = 0
            .then(
                pl.when(
                    calc_missing_count(pl.col("unit_sales")) > 7
                )  # More than 7 missing values
                .then(0)  # --> Impute with 0
                .when(
                    calc_missing_count(pl.col("unit_sales")) <= 7
                )  # if less 7 missing values
                .then(
                    rolling_mean_imputation(pl.col("unit_sales"), window_size)
                )  # --> Inpute with rolling mean for missing 7 or less days
                .otherwise(pl.col("unit_sales"))  # Otherwise keep original value
            )
            .otherwise(pl.col("unit_sales"))  # For any other case not covered
            .alias("unit_sales")  # Alias the new column as 'unit_sales'
        ]
    )

    # Convert Polars df back to Pandas df
    df = df.to_pandas()

    return df

In [13]:
# def impute_stockouts_(df):

#     df = impute_stockouts_polars(df, window_size=7)

#     return df

## 3.2. Aggregate dataset to weekly level

- Group the DataFrame by store number, item number, year, and week_cum_number, then aggregate the columns
--> "unit_sales","onpromotion", "holiday_local_count","holiday_regional_count","holiday_national_count",


In [14]:
def aggregate_week(df):

    # Sort the DataFrame by store number, item number, and date for ordering
    df = df.sort_values(["store_nbr", "item_nbr", "year", "week_nbr"])

    # Group by the specified columns and aggregate
    df = (
        df.groupby(
            [
                "store_nbr",
                "item_nbr",
                "year",
                "week_number_cum",  # Aggregating by week_number_cum
            ]
        )
        .agg(
            {
                "unit_sales": "sum",
                "onpromotion": "sum",
                "holiday_local_count": "sum",
                "holiday_regional_count": "sum",
                "holiday_national_count": "sum",
                "date": "first",  # Keep the first day of week, needed to run Timeseries models from SKtime
                "store_type": "first",  # Keep the first occurrence of store_type
                "store_cluster": "first",  # Keep the first occurrence of store_cluster
                "item_family": "first",  # Keep the first occurrence of item_family
                "item_class": "first",  # Keep the first occurrence of item_class
                "perishable": "first",  # Keep the first occurrence of perishable
                "store_status": "last",  # Keep the last occurrence of store_status
                "item_status": "last",  # Keep the last occurrence of item_status
            }
        )
        .reset_index()
    )

    return df

In [15]:
# df_agg = aggregate_week(df_final)

## X.X Timeseries forecast

## X.1. Drop Unneeded columns, only keep ones for timeseries forecasting
Not a fancy way, later need to think about how to run this in the same pipeline as the ML models

In [16]:
def drop_unnecessary_columns_timeseries(df):

    # List of columns to keep
    columns_to_keep = ["date", "week_number_cum", "store_nbr", "item_nbr", "unit_sales"]

    # Drop all other columns and keep only the specified ones
    df = df[columns_to_keep]

    return df

### X.2.1 Pandas: Creating lagged and rolling mean features for each item-store combination

In [17]:
def add_lagged_features(df, lag_shift=2, rolling_mean_window=3):

    # Sort the DataFrame by store number, item number, and date for ordering
    df = df.sort_values(["store_nbr", "item_nbr", "week_number_cum"])

    # Lag by the specified number of weeks
    df["predicted last"] = df["sales"].shift(lag_shift)

    # Rolling mean (t-2, t-3, t-4)
    df["predicted mean"] = (
        df["sales"].shift(lag_shift).rolling(window=rolling_mean_window).mean()
    )

    # Group by (item_nbr, store_nbr) and apply the lagging and rolling features
    df = (
        df.groupby(["item_nbr", "store_nbr"], group_keys=False)
        .apply(lambda x: x)
        .reset_index(drop=True)
    )

    return df

### X.2.1 Polars: Creating lagged and rolling mean features for each item-store combination

In [18]:
# Function to add lagged features and rolling mean using Polars
def add_lagged_features_timeseries_polars(
    df_pandas, lag_shift=2, rolling_mean_window=3
):

    # Convert the input Pandas DataFrame to a Polars DataFrame
    df = pl.from_pandas(df_pandas)

    # Sort the DataFrame by store number, item number, and date for orderings
    df = df.sort(["store_nbr", "item_nbr", "week_number_cum"])

    # Apply the lag and rolling features
    df = df.with_columns(
        [
            # Lagged sales
            pl.col("unit_sales")
            .shift(lag_shift)
            .alias("predicted last"),  # Shift by 2 weeks for lag
            # Rolling mean of lagged sales
            pl.col("unit_sales")
            .shift(lag_shift)
            .rolling_mean(window_size=rolling_mean_window)
            .alias("predicted mean"),  # Rolling mean (t-2, t-3, t-4)
        ]
    )

    # Convert Polars df back to Pandas df
    df = df.to_pandas()

    return df

testing

In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67834270 entries, 0 to 67834269
Data columns (total 19 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   store_nbr               uint8         
 1   item_nbr                int32         
 2   date                    datetime64[ns]
 3   unit_sales              float32       
 4   onpromotion             bool          
 5   holiday_local_count     int8          
 6   holiday_national_count  int8          
 7   holiday_regional_count  int8          
 8   store_type              category      
 9   store_cluster           uint8         
 10  item_family             category      
 11  item_class              uint16        
 12  perishable              uint8         
 13  store_status            int8          
 14  item_status             int8          
 15  year                    int16         
 16  weekday                 int8          
 17  week_nbr                int8          
 18  week_

In [20]:
def func_functions(df):

    df = impute_stockouts_polars(df, window_size=7)

    df = aggregate_week(df)

    df = drop_unnecessary_columns_timeseries(df)

    df = add_lagged_features_timeseries_polars(df, lag_shift=2, rolling_mean_window=3)

    return df

In [21]:
df_timeseries = func_functions(df_final)

In [22]:
df_timeseries[
    (df_timeseries["item_nbr"] == 105574)  # (df_timeseries["unit_sales"] != 0)
].head(40)

,date,week_number_cum,store_nbr,item_nbr,unit_sales,predicted last,predicted mean
1210,2013-01-02,1,1,105574,26.666666,0.000000,0.000001
1211,2013-01-07,2,1,105574,36.533333,0.000000,0.000001
1212,2013-01-14,3,1,105574,33.000000,26.666666,8.888890
1213,2013-01-21,4,1,105574,38.571426,36.533333,21.066668
1214,2013-01-28,5,1,105574,29.571428,33.000000,32.066669
1215,2013-02-04,6,1,105574,39.571430,38.571426,36.034924
1216,2013-02-11,7,1,105574,27.000000,29.571428,33.714287
1217,2013-02-18,8,1,105574,27.142857,39.571430,35.904766
1218,2013-02-25,9,1,105574,30.333334,27.000000,32.047623
1219,2013-03-04,10,1,105574,34.428570,27.142857,31.238098


In [23]:
def stop

SyntaxError: expected '(' (137546471.py, line 1)

### X.4 Evaulation Metrics

In [ ]:
# Define MAPE function
def mean_absolute_percentage_error(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    nonzero_mask = y_true != 0  # Avoid division by zero
    return (
        np.mean(
            np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])
        )
        * 100
    )

In [ ]:
# Calculate MAPE for the test last 10 observations for each item-store group
mape_results = df.groupby(["item", "store"]).apply(
    lambda group: pd.Series(
        {
            "MAPE Mean": mean_absolute_percentage_error(
                group["sales"].tail(10), group["Predicted Mean"].tail(10)
            ),
            "MAPE Last": mean_absolute_percentage_error(
                group["sales"].tail(10), group["Predicted Last"].tail(10)
            ),
        }
    )
)

# Compute final average MAPE across all item-store combinations
final_mape_mean = mape_results["MAPE Mean"].mean()
final_mape_last = mape_results["MAPE Last"].mean()

print("Final Average MAPE (Mean Forecast): {:.2f}%".format(final_mape_mean))
print("Final Average MAPE (Last Forecast): {:.2f}%".format(final_mape_last))

-------------------------------------------

In [24]:
def evaluate_model(y_true, y_pred):

    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    accuracy = 100 - mape
    bias = np.mean(y_pred - y_true)

    return mape, accuracy, bias

In [44]:
def calculate_metrics_grouped_polars(df):

    # Convert the input Pandas DataFrame to a Polars DataFrame
    df = pl.from_pandas(df)

    def calculate_group_metrics(group):
        y_true = group.filter(pl.col("unit_sales") != 0)[
            "unit_sales"
        ]  # Avoid division by zero for MAPE
        metrics = []

        for col in ["predicted_last", "predicted_mean"]:
            y_pred = group.filter(pl.col("unit_sales") != 0)[
                col
            ]  # Avoid division by zero for MAPE

            # Use evaluate_model function to get metrics
            mape, accuracy, bias = evaluate_model(y_true.to_numpy(), y_pred.to_numpy())

            metrics.append(
                pl.DataFrame(
                    {
                        "prediction_type": [col],
                        "MAPE": [mape],
                        "Accuracy": [accuracy],
                        "Bias": [bias],
                    }
                )
            )

        return pl.concat(metrics)

    # Group by 'item_nbr' and 'store_nbr' and calculate metrics
    metrics_df = (
        df.group_by("store_nbr", "item_nbr")
        .apply(calculate_group_metrics)
        .explode(["prediction_type", "MAPE", "Accuracy", "Bias"])
    )

    # Calculate the final average MAPE across all item-store combinations
    final_average_mape = metrics_df["MAPE"].mean()

    print(
        f"Final average MAPE across all item-store combinations: {final_average_mape:.2f}%"
    )
    # print(grouped_metrics)

    return metrics_df

In [48]:
# Function to evaluate metrics on Polars DataFrame columns grouped by item and store
def calculate_metrics_grouped_polars(df):
    # Convert the input Pandas DataFrame to a Polars DataFrame
    df = pl.from_pandas(df)

    grouped_metrics = []

    # Group by 'item_nbr' and 'store_nbr' and iterate through each group
    for group in df.groupby(["store_nbr", "item_nbr"]):
        store = group["store_nbr"][0]
        item = group["item_nbr"][0]

        # Filter out rows where 'unit_sales' is zero to avoid division errors in MAPE
        filtered_group = group.filter(pl.col("unit_sales") != 0)
        y_true = filtered_group["unit_sales"]

        for col in ["predicted_last", "predicted_mean"]:
            y_pred = filtered_group[col]

            # Use evaluate_model function to get metrics
            mape, accuracy, bias = evaluate_model(y_true.to_numpy(), y_pred.to_numpy())

            grouped_metrics.append(
                {
                    "store_nbr": store,
                    "item_nbr": item,
                    "prediction_type": col,
                    "MAPE": mape,
                    "Accuracy": accuracy,
                    "Bias": bias,
                }
            )

    # Convert the metrics list to a Polars DataFrame
    metrics_df = pl.DataFrame(grouped_metrics)

    # Calculate the final average MAPE across all item-store combinations
    final_average_mape = metrics_df["MAPE"].mean()

    print(
        f"Final average MAPE across all item-store combinations: {final_average_mape:.2f}%"
    )

    return metrics_df

In [49]:
metrics_df = calculate_metrics_grouped_polars(df_timeseries)

AttributeError: 'DataFrame' object has no attribute 'groupby'

## 4. Column transformers

### 4.1. Column transformers

In [ ]:
features = [
    "store_nbr",
    "item_nbr",
    "onpromotion",
    "holiday_local_count",
    "holiday_national_count",
    "holiday_regional_count",
    "store_type",
    "store_cluster",
    "item_family",
    "item_class",
    "perishable",
    "store_status",
    "item_status",
    "year",
    "week_nbr",
    "week_number_cum",
]

target_variable = ["unit_sales"]

X = df[features]
y = df[target_variable]

To-do: Do we need onehotencoder? --> then needed to seperate between timeseries en ML models

To-do: Change catagory dtypes from store_type and item_family just to numbers in prep pipeline?

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# Create your transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        (
            "ImputeStockouts",  # Impute stockouts
            FunctionTransformer(impute_stockouts_polars),
            num_features.tolist() + cat_features.tolist(),
        ),
        (
            "AggregateWeek",  # Aggregate dataset to weekly level
            FunctionTransformer(aggregate_week_polars),
            num_features.tolist() + cat_features.tolist(),
        ),
        ("StandardScaler", StandardScaler(), num_features),  # To-do: --> needed?????
    ]
)

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

## 5. Models

### 5.1. Models list to compare in model

In [ ]:
# <PATH>.\venv_case_project\Scripts\activate

# source venv_macbook/bin/activate

# pip install sktime
# pip install statsmodels
# pip install xgboost

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

models = {
    "Naive": NaiveForecaster(strategy="last"),
    "Simple Moving Average": NaiveForecaster(strategy="mean", window_length=6),
    "Holt-Winters": ExponentialSmoothing(trend="add", seasonal="add", sp=52),
    "Random Forest Regressor": make_reduction(
        RandomForestRegressor(), window_length=13, strategy="recursive"
    ),
    "XGBoost": make_reduction(XGBRegressor(), window_length=13, strategy="recursive"),
}

### 5.2. Evaulation Metrics and Evaluate Model functions

In [1]:
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    r2_score,
)


def forecast_accuracy(y_true, y_pred):

    # Calculate the absolute differences between true and predicted values
    absolute_errors = np.abs(y_true - y_pred)

    # Calculate 10% of the absolute true values
    tolerance = 0.1 * np.abs(y_true)

    # Check if the absolute errors are within 10% of the true values
    within_tolerance = absolute_errors <= tolerance

    # Calculate and return the mean of the boolean array
    # (True is treated as 1 and False as 0), which gives the proportion of accurate forecasts
    return np.mean(within_tolerance)


def evaluate_model(y_true, y_pred):

    mape = mean_absolute_percentage_error(y_true, y_pred)
    accuracy = forecast_accuracy(y_true, y_pred)
    bias = np.mean(y_true - y_pred)

    return mape, accuracy, bias

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score


def evaluate_forecast(y_true, y_pred, tolerance=0.1):

    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # Calculate absolute errors and tolerance
    absolute_errors = np.abs(y_true - y_pred)
    tolerance_values = tolerance * np.abs(y_true)

    # Calculate metrics
    mape = mean_absolute_percentage_error(y_true, y_pred)
    bias = np.mean(y_true - y_pred)

    # Calculate accuracy (within tolerance)
    within_tolerance = absolute_errors <= tolerance_values
    accuracy = np.mean(within_tolerance)

    # Calculate direction accuracy
    direction_correct = np.sign(y_true[1:] - y_true[:-1]) == np.sign(
        y_pred[1:] - y_pred[:-1]
    )
    direction_accuracy = np.mean(direction_correct)

    return {
        "MAPE": mape,
        "Bias": bias,
        "Accuracy (within {}% tolerance)".format(tolerance * 100): accuracy,
        "Direction Accuracy": direction_accuracy,
    }

Run / Fit model

Test Claude

In [ ]:
from sktime.performance_metrics.forecasting import (
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
)


def evaluate_models(models, y_train, y_val, y_test, fh):
    results = []

    for model_name, model in models.items():
        model.fit(y_train)

        y_val_pred = model.predict(fh[: len(y_val)])
        y_test_pred = model.predict(fh)

        val_mape, val_accuracy, val_bias = evaluate_model(y_val, y_val_pred)
        test_mape, test_accuracy, test_bias = evaluate_model(y_test, y_test_pred)

        results.append(
            {
                "Model": model_name,
                "Validation MAPE": val_mape,
                "Validation Accuracy": val_accuracy,
                "Validation Bias": val_bias,
                "Test MAPE": test_mape,
                "Test Accuracy": test_accuracy,
                "Test Bias": test_bias,
            }
        )

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values("Test MAPE")

    return results_df


# Usage example:
# Assuming you have your data split into y_train, y_val, y_test
# and a forecast horizon fh defined

# models = create_models()
# results = evaluate_models(models, y_train, y_val, y_test, fh)
# print(results)

## X. Pick best one --> Optimize with grid search

In [ ]:
# 	Get	feature	importances	from	the	model
feature_importances = best_model.get_feature_importance(prettified=False)

# 	Get	feature	names	(considering	potential	transformation)
feature_names = preprocessor.get_feature_names_out()  # 	After	column	transformation

# 	Sort	feature	importances	and	names	together	by	importance	(descending)
sorted_idx = np.argsort(feature_importances)
feature_importances = feature_importances[sorted_idx]
feature_names = feature_names[sorted_idx]

# 	Define	plot	size	and	create	a	bar	chart
plt.figure(figsize=(12, 6))
plt.barh(range(len(feature_names)), feature_importances, align="center")
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel("Feature	Importance")
plt.ylabel("Feature	Names")
plt.title("Feature	Importance	for	Electricity	Demand-Supply	Prediction")
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1, 0.2],
    "iterations": [50, 100, 200],
}

best_model = CatBoostRegressor()

grid_search = GridSearchCV(estimator=best_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [ ]:
best_param

In [ ]:
param_grid = {
    "svm__C": [0.001, 0.01, 0.1, 1, 10, 100],
    "svm__gamma": [0.001, 0.01, 0.1, 1, 10, 100],
}
pipe = pipeline.Pipeline([("scaler", MinMaxScaler()), ("svm", SVC(C=100))])
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

Nested cross-validation
https://ml-course.github.io/master/notebooks/Tutorial%203%20-%20Machine%20Learning%20in%20Python.html#evaluate

In [ ]:
scores = cross_val_score(
    GridSearchCV(SVC(), param_grid, cv=5), iris.data, iris.target, cv=5
)

In [ ]:
scores = cross_val_score(
    GridSearchCV(SVC(), param_grid, cv=5), iris.data, iris.target, cv=5
)
print("Cross-validation scores: ", scores)
print("Mean cross-validation score: ", scores.mean())

In [ ]:
df_eda.to_csv("final_model.csv", index=False)
# 	Save	the	trained	model
lr_model.save_model("catboost_model.cbm")

To-do: Residual analysis?
--> Check if errors are randomly distributed in pointcloud